In [7]:
class Node:
    def __init__(self):
        self.terminal = None
        self.children = dict()
    
class BOR:
    def __init__(self):
        self.root = Node()
    
    def b_insert(self, word, cur_node):
        for symb in word:
            if symb not in cur_node.children:
                cur_node.children[symb] = Node()
            cur_node = cur_node.children[symb]
        
        cur_node.terminal = word
    
    def insert(self, word):
        self.b_insert(word, self.root)
    
    def b_nearest_words(self, cur_node, word, max_dist, prev_lev, symb, res):
        n2 = len(word) + 1
        lev = []
        
        lev.append(prev_lev[0] + 1)
        
        for j in range(1, n2):
            var1 = lev[j-1]+1
            var2 = prev_lev[j]+1
            var3 = prev_lev[j-1]
            if symb != word[j-1]:
                var3 += 1
            lev.append(min(var1, var2, var3))
            
        if cur_node.terminal is not None and lev[n2-1] <= max_dist:
            res.append([cur_node.terminal, lev[n2-1]])
            
        flag = False
        for elt in lev:
            if elt <= max_dist:
                flag = True
                break
            
        if flag:
            for symb in cur_node.children:
                self.b_nearest_words(cur_node.children[symb], word, max_dist, lev, symb, res)
                
    def nearest_words(self, word, max_dist):
        n1 = len(word)+1
        lev = []
        for i in range(n1):
            lev.append(i)
        res = []
        for symb in self.root.children:
            self.b_nearest_words(self.root.children[symb], word, max_dist, lev, symb, res)
        return res    

In [8]:
class ErrorModel():
    
    def __init__(self, alpha = 0.5):
        self.alpha = alpha
        self.tree = BOR()
        
    def add_to_bor(self, word):
        self.tree.insert(word)
        
    def lev_dist(s1, s2):
        n1 = len(s1)+1
        n2 = len(s2)+1
        lev = []
        for i in range(n1):
            lev.append([0]*n2)
        for i in range(1, n1):
            lev[i][0] = i
        for i in range(1, n2):
            lev[0][i] = i
        for i in range(1, n1):
            for j in range(1, n2):
                var1 = lev[i-1][j]+1
                var2 = lev[i][j-1]+1
                var3 = lev[i-1][j-1]
                if s1[i-1] != s2[j-1]:
                    var3 += 1
                lev[i][j] = min(var1, var2, var3)
        return lev[n1-1][n2-1]
    
    def get_nearest(self, word, max_dist):
        nearest = self.tree.nearest_words(word, max_dist)
        res = []
        for elt in nearest:
            res.append([elt[0], self.alpha ** (-elt[1])])
        return res

In [13]:
class LanguageModel():
    
    def __init__(self):
        self.stat = dict()
        self.stat2 = dict()
        self.size = 0
        self.size2 = 0
        self.Dict = set()
        
    def add_to_dict(self, word):
        self.Dict.add(word)
    
    def isin(self, word):
        return word in Dict
        
    def add_to_stat(self, word):
        self.size += 1
        if word in self.stat:
            self.stat[word] += 1
        else:
            self.stat[word] = 1
            
    def add_to_stat2(self, word):
        self.size2 += 1
        if word in self.stat2:
            self.stat2[word] += 1
        else:
            self.stat2[word] = 1
    
    def to_bigram(self, words):
        res = []
        for i in range(1, len(words)):
            bigram = words[i-1] + words[i]
            res.append(bigram)
        return res
    
    def get_freq_query(self, words):
        p = 1
        for word in words:
            if word in self.stat:
                p *= self.stat[word]/self.size
        return p
    
    def get_freq_query2(self, words):
        p = 1
        for i in range(1, len(words)):
            bigram = words[i-1] + words[i]
            if bigram in self.stat2:
                p *= self.stat2[bigram]/self.size2
        return p
    
    def get_frequency_word(self, word):
        if word in self.stat:
            return self.stat[word]/self.size
        else:
            return 0
    
    def delete_rare_words(self, split):
        for key in self.stat:
            if get_frequency(key) < split:
                self.stat.pop(key)

In [14]:
def get_features(words):
    length = 0
    for elt in words:
        length += len(elt)
    k_words = len(words)
    if k_words > 1:
        freq = LangModel.get_freq_query2(words)
    else:
        freq = LangModel.get_freq_query(words)
    
    in_dict = 0
    max_p = 0
    min_p = 1
    
    for word in words:
        if word in Dict:
            in_dict += 1
        p = LangModel.get_frequency_word(word)
        min_p = min(p, min_p)
        max_p = max(p, max_p)
    
    not_in_dict = k_words - in_dict
    
    return [length, k_words, not_in_dict, max_p, min_p, freq]

In [15]:
import re

def get_before_tab(s):
    return s[:s.find('\t')]

def get_after_tab(s):
    return s[s.find('\t')+1:]

f = open('queries_all.txt', 'r')
#spec = ' ?!(),.+'
spec = ' '

ErrModel = ErrorModel(2)
LangModel = LanguageModel()
train_x = []
train_y = []

for line in f:
    if line[-1] == '\n':
        line = line[:-1]
    orig = get_before_tab(line)
    fix = get_after_tab(line)
    words = fix.split(spec)
    #words = [re.sub(r'[^A-zА-я0-9]', '', word) for word in words] 
    if fix != "":
        for word in words:
            LangModel.add_to_stat(word) 
            LangModel.add_to_dict(word)
            ErrModel.add_to_bor(word)
        bi_words = LangModel.to_bigram(words)
        if bi_words:
            for word in bi_words:
                LangModel.add_to_stat2(word)
        train_x.append(get_features(words))
        train_y.append(1)
        train_x.append(get_features(orig.split(spec)))
        train_y.append(0)
    else:
        train_x.append(get_features(orig.split(spec)))
        train_y.append(0)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

size = 300000
subtrain_x = train_x[:size]
subtrain_y = train_y[:size]

subtest_x = train_x[size:]
subtest_y = train_y[size:]

#clf = GradientBoostingClassifier()
clf = LGBMClassifier()
clf.fit(subtrain_x, subtrain_y)
pred = clf.predict(subtest_x)
f1_score(pred, subtest_y)

0.8871379693735069

In [16]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier()
clf.fit(train_x, train_y)

LGBMClassifier()

In [38]:
from itertools import product

rus2eng = {'й': 'q', 'ц': 'w', 'у': 'e', 'к': 'r', 'е': 't', 'н': 'y', 'г': 'u', 'ш': 'i', 'щ': 'o', 'з': 'p', 'х': '[', 'ъ': ']', 
           'ф': 'a', 'ы': 's', 'в': 'd', 'а': 'f', 'п': 'g', 'р': 'h', 'о': 'j', 'л': 'k', 'д': 'l', 'ж': ';', 'э': "'", 
           'я': 'z', 'ч': 'x', 'с': 'c', 'м': 'v', 'и': 'b', 'т': 'n', 'ь': 'm', 'б': ',', 'ю': '.' }
eng2rus = {'q': 'й', 'w': 'ц', 'e': 'у', 'r': 'к', 't': 'е', 'y': 'н', 'u': 'г', 'i': 'ш', 'o': 'щ', 'p': 'з', '[': 'х', ']': 'ъ', 
           'a': 'ф', 's': 'ы', 'd': 'в', 'f': 'а', 'g': 'п', 'h': 'р', 'j': 'о', 'k': 'л', 'l': 'д', ';': 'ж', "'": 'э', 
           'z': 'я', 'x': 'ч', 'c': 'с', 'v': 'м', 'b': 'и', 'n': 'т', 'm': 'ь', ',': 'б', '.': 'ю' }

def change_keyboard(words):
    res = []
    for word in words:
        changed = ""
        for symb in word:
            if symb in rus2eng:
                changed += rus2eng[symb]
            elif symb in eng2rus:
                changed += eng2rus[symb]
            else:
                changed += symb
        res.append(changed)
    return res

def change_split(words):
    res = []
    num = 0
    for word in words:
        if not LangModel.isin(word):
            for ind in range(1, len(word)-1):
                split = []
                split.extend(words[:num])
                split.append(word[0:ind])
                split.append(word[ind:])
                split.extend(words[num+1:])
                res.append(split)
        num += 1
    return res

def get_corrections(words):
    res = []
    for word in words:
        if LangModel.isin(word):
            res.append([word])
        else:
            nearest = ErrModel.get_nearest(word, 1)
            res.append([])
            if len(nearest) > 0:
                for w, value in nearest:
                    res[-1].append(w)
            else:
                res[-1].append(word)
    return product(*res)

In [58]:
import timeit
k_iter = 2

while True:
    %timeit
    query = input()
    if query[-1] == '\n':
        query = query[:-1]
    words = query.split(spec)
    features = [get_features(words)]
    if clf.predict(features) == 1:
        print(query)
    else:
        flag = False
        for i in range(k_iter):
            variants = []
            features = []
            prob = []
            
            arr = get_corrections(words)
            for q in arr:
                variants.append(list(q))
                features.append(get_features(q))
                prob.append(LangModel.get_freq_query2(q))
            
            arr = change_keyboard(words)
            variants.append(arr)
            features.append(get_features(arr))
            prob.append(LangModel.get_freq_query2(arr))
            
            arr = change_split(words)
            if len(arr) != 0:
                #variants.extend(arr)
                for q in arr:
                    variants.append(q)
                    features.append(get_features(q))
                    prob.append(LangModel.get_freq_query2(q))
            max_freq = -1
            num = -1
            all_max_freq = -1
            all_num = -1
            for i in range(len(variants)):
                if clf.predict([features[i]]) == 1:
                    if prob[i] > max_freq:
                        max_freq = prob[i]
                        num = i
                if prob[i] > all_max_freq:
                    all_max_freq = prob[i]
                    all_num = i
            if num != -1:
                print(' '.join(variants[num]))
                flag = True
                break
            else:
                words = variants[all_num]
                
        if not flag:
            print(' '.join(words))

привет
привет
прувет
привет
превет
привет
прувет как дила
прувер как дила
привет как дело
ghbdtn rfr lkj
ghbdtn
привет
приветик
приветик
купить машину
купить машину
купить машыну
купить машыну
купить машына
купить машина


KeyboardInterrupt: Interrupted by user

In [67]:
import pickle

def save(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)


def load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [68]:
mkdir objects

In [69]:
save(clf, 'objects/Classificator.pkl')

In [ ]:
import sys
sys.setrecursionlimit(100000)
save(ErrModel, 'objects/ErrorModel.pkl')

In [71]:
save(LangModel, 'objects/LanguageModel.pkl')

KeyboardInterrupt: 